In [ ]:
%matplotlib nbagg
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import interpolate
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm import tqdm

import pickle

import linear_operators as lin_ops
import boundary_conditions as bcs
import time_steppers as tstep
import rom_functions as rom

import time as tlib


plt.rcParams.update({"text.usetex":True,\
                     "font.family":"serif",\
                     "font.sans-serif":["Computer Modern"]})

# Jet flow setup

In [ ]:
class jet_class:
    
    def __init__(self,r,z,Re,theta0):
        
        self.r = r
        self.z = z
        self.rowsu = len(r) + 2
        self.colsu = len(z) + 1
        self.szu = self.rowsu*self.colsu
        self.rowsv = len(r) + 1
        self.colsv = len(z) + 2
        self.szv = self.rowsv*self.colsv
        self.rowsp = len(r)
        self.colsp = len(z)
        self.szp = self.rowsp*self.colsp + 1
        
        self.Re = Re
        self.theta0 = theta0
        self.q_sbf = 0

def assemble_initial_conditions(jet):
    
    qic = np.zeros(jet.szu + jet.szv)
    for i in range (1,jet.rowsu-1):
        for j in range (1,jet.colsu):
            
            k = i*jet.colsu + j
            qic[k] = 0.5*(1 - np.tanh((jet.r[i-1] - 1/(4*jet.r[i-1]))/(4*jet.theta0)))
    
    qic = bcs.update_boundary_conditions(jet.r,jet.z,qic,jet.theta0,1)
    return qic




def generate_data(jet,lops,time,nsave,alphas):
    # jet & lops:   cfd classes
    # q_sbf:        steady base flow
    # time:         time vector (this specifies the cfd time stepper)
    # nsave:        save data every nsave snapshots
    # alphas:       impulse magnitude
    
    
    # Outputs:  x_traj_data with size (see below)
    #           t_data = time[::nsave]
    
    N = lops.Mrem.shape[0]
    x_traj_data = np.zeros((len(alphas),N,len(time)//nsave),dtype=np.float64)
    for k in range (len(alphas)):
        print("Simulating trajectory %d/%d with alpha = %1.3e"%(k+1,len(alphas),alphas[k]))
        qic = lops.B*alphas[k] + jet.q_sbf
        x, t_data = tstep.nonlinear_solver(jet,lops,time,nsave,qic)
        x = lops.Wsqrt.dot(x - jet.q_sbf.reshape((jet.szu+jet.szv,1)))
        x_traj_data[k,] = lops.Mrem.dot(x)
        
    return x_traj_data, t_data


def simulate_nonlinear_forward(jet,lops,x_ic,time,nsave,*argv):
    q_ic = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_ic))) + jet.q_sbf
    
    if len(argv) == 0:
        data_nonlin, tsave_nonlin = tstep.nonlinear_solver(jet,lops,time,nsave,q_ic)
    else:
        forcing = argv[0]
        data_nonlin, tsave_nonlin = tstep.nonlinear_solver(jet,lops,time,nsave,q_ic,forcing)
        
    data_nonlin = lops.Mrem.dot(lops.Wsqrt.dot(data_nonlin - jet.q_sbf.reshape((-1,1))))
    
    return data_nonlin, tsave_nonlin


def simulate_linear_forward(jet,lops,v_ic,time,nsave,*argv):
    # v_ic:         initial condition in "x" coordinates x = sqrt(W)(q - q_sbf)
    # q_sbf:        this is the q-coordinates steady state (usual thing)
    
    if len(argv) > 0:       # this lets you linearize about time-varying base flow
        xbaseflow = argv[0]
        # Take a euclidean vec qic with no boundary points. Transform to weighted
        # statespace and add boundaries
        qbaseflow = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(xbaseflow))) + jet.q_sbf.reshape((-1,1))
        flag = 'ubf'
    else:                   # this is linerization about steady base flow
        qbaseflow = jet.q_sbf
        flag = 'sbf'
    
    vq_ic = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(v_ic)))
    data_lin, tsave_lin = tstep.linear_solver(jet,lops,time,nsave,qbaseflow,vq_ic,flag)
    data_lin = lops.Mrem.dot(lops.Wsqrt.dot(data_lin))
    
    return data_lin, tsave_lin


def simulate_linear_adjoint(jet,lops,lam_fc,time,nsave,*argv):
    
    
    if len(argv) > 0:       # this lets you linearize about time-varying base flow
        xbaseflow = argv[0]
        # Take a euclidean vec qic with no boundary points. Transform to weighted
        # statespace and add boundaries
        qbaseflow = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(xbaseflow))) + jet.q_sbf.reshape((-1,1))
        flag = 'ubf'
    else:                   # this is linerization about steady base flow
        qbaseflow = jet.q_sbf
        flag = 'sbf'
    
    lam_q_fc = lops.BC_op_T.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(lam_fc)))
    data_adj, t_adj = tstep.adjoint_solver(jet,lops,time,nsave,qbaseflow,lam_q_fc,flag)
    data_adj = np.fliplr(lops.Mrem.dot(lops.Wsqrt.dot(data_adj)))
    
    return data_adj, t_adj

def assemble_rom(jet,lops,Phi,Psi):
    f0_ROM = np.dot(Psi.T, rom.evaluate_f(jet,lops,0.0*Phi[:,0],0.0))
    
    B_ROM = np.reshape(np.dot(Psi.T, rom.evaluate_f(jet,lops,0.0*Phi[:,0],1.0)) - f0_ROM, (-1,1))
    
    C_ROM = Phi
    
    A_ROM = np.zeros((r, r))
    for j in range(r):
        A_ROM[:,j] = np.dot(Psi.T, rom.evaluate_df(jet,lops,0.0*Phi[:,0],0.0,Phi[:,j]))
    
    N_ROM = np.zeros((r, 1, r))
    
    H_ROM = np.zeros((r, r, r))
    for i in tqdm(range(r), position=0, leave=True, desc="assembling 3rd order tensor"):
        H_ROM[:,i,i] = np.dot(Psi.T, rom.evaluate_f(jet,lops,Phi[:,i],0.0)) - f0_ROM - A_ROM[:,i]
        for j in range(1, r):
            H_ROM[:,i,j] = np.dot(Psi.T, rom.evaluate_f(jet,lops,Phi[:,i]+Phi[:,j],0.0) \
                              - rom.evaluate_f(jet,lops,Phi[:,i]-Phi[:,j],0.0) )/4.0 - A_ROM[:,j]/2.0
            H_ROM[:,j,i] = H_ROM[:,i,j]
    
    return f0_ROM, A_ROM, B_ROM, C_ROM, H_ROM, N_ROM

def output_fields(jet,q):
    
    u = np.zeros((jet.rowsp+1,jet.colsp+1))
    v = u.copy()
    w = u.copy()
    
    dr = jet.r[1] - jet.r[0]
    dz = jet.z[1] - jet.z[0]
    
    r = np.zeros(jet.rowsp+1)
    z = np.zeros(jet.colsp+1)
    for j in range (jet.rowsp+1): r[j] = j*dr
    for j in range (jet.colsp+1): z[j] = j*dz
    Z, R = np.meshgrid(z,r)
    
    for i in range (0,jet.rowsp+1):
        for j in range (0,jet.colsp+1):
            
            ku = i*jet.colsu + j
            kv = i*jet.colsv + j + jet.szu
            
            u[i,j] = 0.5*(q[ku] + q[ku+jet.colsu])
            v[i,j] = 0.5*(q[kv] + q[kv+1])
            w[i,j] = (q[kv+1] - q[kv])/dz - (q[ku+jet.colsu] - q[ku])/dr
            w[i,j] = w[i,j]
            
    return u, v, w, Z, R

## Jet flow parameters

In [ ]:
# ---------------------------------------------------
# --------- Assemble grid ---------------------------
# ---------------------------------------------------

Lr = 4              # Length of radial direction
Nr = 200            # Number of grid points in radial direction
dr = Lr/Nr
r = np.zeros(Nr)
for j in range (0,Nr): r[j] = j*dr + dr/2


Lz = 10             # Length of axial direction
Nz = 250            # Number of grid points in axial direction
dz = Lz/Nz
z = np.zeros(Nz)
for j in range (0,Nz): z[j] = j*dz + dz/2


Re = 2000           # Reynolds number
theta0 = 0.025      # non-dimensional thickness of incoming flow 


r0 = 0.5            # Radial location of center of forcing Gaussian blob
z0 = 1.0            # Axial location of center of forcing Gaussian blob
direction = 'radial'# The forcing impulse enters the radial momentum equation (can change to 'axial')
jet = jet_class(r,z,Re,theta0)  
lops = lin_ops.linear_operators_class(jet,r0,z0,direction)

# ---------------------------------------------------
# --------- Specify time horizon --------------------
# ---------------------------------------------------
dt = 0.005           # dt for simulation
n = np.intc(1/dt)    # number of time steps per non-dimensional time unit
N = 50*n             # total number of time steps (i.e., tf = dt*N) # 30*n for Re=1000, 50*n for Re=2000
time = dt*np.arange(0,N,1)
nsave = 100          # save data every nsave snapshots

## Generate base flow

In [ ]:
# qic = assemble_initial_conditions(jet)
# data, _ = tstep.nonlinear_solver(jet,lops,time,nsave,qic)
# data, _ = tstep.nonlinear_solver(jet,lops,time,nsave,data[:,-1])
# data, _ = tstep.nonlinear_solver(jet,lops,time,nsave,q_sbf)

In [ ]:
# np.savetxt("steady_state_solution_Re2000_Nr200_Nz250.txt",data[:,-1])
# np.savetxt("data/steady_state_solution_Re2000_Nr200_Nz250.txt", data[:,-1])

## Load base flow

In [ ]:
# q_sbf = np.loadtxt("steady_state_solution_Re2000_Nr200_Nz250.txt") # Load the stable steady state
q_sbf = np.loadtxt("data/steady_state_solution_Re2000_Nr200_Nz250.txt")
jet.q_sbf = q_sbf

# Generate data

## Impulse-response data

In [ ]:
alphas = np.asarray([0.005, 0.02, 0.04, 0.06, 0.08, 0.10]) # training data
alphas = np.concatenate((-np.flipud(alphas),alphas))

# alphas = np.random.uniform(low=-0.10, high=0.10, size=(25)) # evaluation and testing data

x_traj_data, t_data = generate_data(jet,lops,time,nsave,alphas)

### Save data

In [ ]:
fname = 'data/traj_data_train.mat'
data = {'alphas': alphas, \
        'x_traj_data': x_traj_data, \
        't_data': t_data, \
        'q_sbf': q_sbf}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

### Load data

In [ ]:
fname = 'data/traj_data_train.mat'

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

alphas = data['alphas']
x_traj_data = data['x_traj_data']
t_data = data['t_data']
n_traj = len(alphas)
u_traj = np.zeros((n_traj, 1, len(time)))

## Energy along trajectories

In [ ]:
plt.figure()
energy = np.zeros((len(alphas),len(t_data)))
for k in range (len(alphas)):
    for j in range (len(t_data)):
        energy[k,j] = np.dot(x_traj_data[k,:,j],x_traj_data[k,:,j])
        
    plt.plot(t_data, energy[k,],'k')

# plt.savefig('../figures/jet_flow/energy_trajectories.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/energy_trajectories.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

## Plot a flowfield snapshot

In [ ]:
# Add boundary conditions to the flowfield you wanna plot. This flowfield is a perturbation 
# about the base flow q_sbf

t_idx = 60
traj_idx = 0

flowfield = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf

# flowfield = q_sbf

#flowfield = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(Phi_cbal[:,-1])))


# u, v, w, Z, R = output_fields(jet,jet.q_sbf)

u, v, w, Z, R = output_fields(jet,flowfield)

field = w
color_map = 'hot_r'

vminval = -np.max(field)*0
vmaxval = np.max(field)

plt.figure()
plt.contourf(Z,R,w,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
ax = plt.gca()
ax.set_aspect(1)
ax.set_ylim([0,1.5])
ax.set_xlabel('z',fontsize=14)
ax.set_ylabel('r',fontsize=14)

m = plt.cm.ScalarMappable(cmap=color_map)
m.set_array(field)
m.set_clim(vminval,vmaxval)

cbar = plt.gcf().colorbar(m,orientation='horizontal',pad=0.2)
cbar.ax.set_xlabel('Azimuthal vorticity',fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Covariance Balancing

## Gradient Sampling

In [ ]:
# Initialize large vectors (for efficiency)
eta = np.zeros(lops.Mrem.shape[0])
lam_fc = eta.copy()
x0 = eta.copy()

# horizon length
L = 40
delta_t_samples = t_data[1]-t_data[0]

# number of samples per trajectories
#n_samples = len(t_data)*(L+1) # 100
n_samples = 10 # at random
i0 = 0 # initial sample index (use for adding to an existing set of samples)
n_traj = len(alphas)

wclock0 = tlib.time()
for k in range(n_traj):
    
    print("Current trajectory: %d"%(k+1))
    
    ### construct gradient sample matrix
    
    # generate random final time samples
    tf_idx_samples = np.zeros(n_samples, dtype=np.intp)
    for i in range(n_samples):
        t_idx = np.random.randint(0, len(t_data))
        tau_idx = np.random.randint(0, L)
        tf_idx_samples[i] = t_idx + tau_idx
    
#    # generate deterministic final time samples
#    tf_idx_samples = np.zeros(n_samples, dtype=np.intp)
#    for i in range(len(t_data)):
#        for j in range(L+1):
#            tf_idx_samples[i*(L+1)+j] = i + j
    
    # loop through samples
    for i in range(n_samples):
        
        print("Current sample: %d"%(i+1))
        ## define the mini-trajectory
        
        # final time index
        tf_idx = tf_idx_samples[i]
        
        # overlap with base trajectory (t = tf - tau)
        tau_idx_min = max(0, tf_idx-len(t_data))
        tau_idx_max = min(L, tf_idx)
        nu = 1 + tau_idx_max - tau_idx_min
        tau_idx_samples = np.arange(tau_idx_min, tau_idx_max+1)
        tau_samples = tau_idx_samples * delta_t_samples
        
        # initial time index
        t0_idx = tf_idx - tau_idx_max
        
        # initial and final times for mini-traj
        t0 = t_data[t0_idx]
        tf = t0 + delta_t_samples*(tf_idx - t0_idx)
        
        ## simulate forward along mini-trajectory
        x0[:] = x_traj_data[k,:,t0_idx]
        
        if t0_idx < tf_idx:
            
            tspan = np.linspace(t0,tf,nsave*(tf_idx - t0_idx)+1)
            sol_nonlin, _ = simulate_nonlinear_forward(jet,lops,x0,tspan,1)
            
            # random isotropic vector
            eta[:] = np.sqrt(L+1)*np.random.randn(sol_nonlin.shape[0])
            lam_fc[:] = eta
            
            
            sol_adj, _ = simulate_linear_adjoint(jet,lops,lam_fc,tspan,nsave,sol_nonlin)
            
            # extract adjoint samples that overlap with data
            sol_adj = sol_adj[:,:nu]
            
            Y_samples = sol_adj / np.sqrt(nu)
            
            x_samples = sol_nonlin[:,::nsave]
            x_Y_samples = x_samples[:,:nu]
            
            del sol_nonlin
            
        else:
            # random isotropic vector
            eta[:] = np.sqrt(L+1)*np.random.randn(sol_nonlin.shape[0])
            lam_fc[:] = eta
            
            Y_samples = np.reshape(lam_fc, (-1,1))
            
            x_Y_samples = np.reshape(x0, (-1,1))
        
        fname = 'data/cbal_L{:d}_traj{:d}_sample{:d}.mat'.format(L, k, i0+i)
        data = {'Y_samples': Y_samples, \
                'x_Y_samples': x_Y_samples}
        
        with open(fname, 'wb') as fh:
           pickle.dump(data, fh)
        
        del Y_samples, x_Y_samples, data

wclock1 = tlib.time()
print("Total runtime: %1.3e"%((wclock1-wclock0)/60))

## Load gradient samples

In [ ]:
# horizon length
L = 40

# number of samples per trajectories
#n_samples = len(t_data)*(L+1) # 100
n_samples = 10
n_traj = len(alphas)

# gradient sample matrix
Y_samples = []
# states at gradient samples
x_Y_samples = []
for k in range(n_traj):
    print('loading samples from trajectory {:d} of {:d}\n'.format(k+1, n_traj))
    for i in range(n_samples):
        fname = 'data/cbal_L{:d}_traj{:d}_sample{:d}.mat'.format(L, k, i)
        with open(fname, 'rb') as fh:
           data = pickle.load(fh)
        
        Y_samples = Y_samples + [data['Y_samples']]
        x_Y_samples = x_Y_samples + [data['x_Y_samples']]

# stack adjoint samples
Y = np.hstack(Y_samples) / np.sqrt(n_traj*n_samples)
del Y_samples

x_Y = np.hstack(x_Y_samples)
del x_Y_samples

### state sample matrix
X = np.hstack(x_traj_data)/np.sqrt(n_traj*len(t_data))

## Save covariance balancing data

In [ ]:
fname = 'data/cbal_L{:d}_{:d}trajs_by_{:d}samples.mat'.format(L, n_traj, n_samples)
data = {'X': X, \
        'Y': Y, \
        'x_Y': x_Y}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

## Load covariance balancing data

In [ ]:
L = 40

n_samples = 10
n_traj = 12

fname = 'data/cbal_L{:d}_{:d}trajs_by_{:d}samples.mat'.format(L, n_traj, n_samples)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

X = data['X']
Y = data['Y']
x_Y = data['x_Y']

del data

## construct projection

In [ ]:
U_cbal, s_cbal, VT = sp.linalg .svd(Y.T@X)
V_cbal = VT.T

var_frac_cbal = np.cumsum(np.square(s_cbal))/np.sum(np.square(s_cbal))

In [ ]:
plt.figure()
plt.semilogy(np.arange(len(s_cbal))+1, 1.0 - var_frac_cbal, 'ko')
plt.show()

In [ ]:
r = 40
Phi_cbal = X@V_cbal[:,:r]/np.sqrt(s_cbal[:r])
Psi_cbal = Y@U_cbal[:,:r]/np.sqrt(s_cbal[:r])

# construct reduced-order model
f0_cbal, A_cbal, B_cbal, C_cbal, H_cbal, N_cbal = assemble_rom(jet,lops, Phi_cbal, Psi_cbal)

def cbal_ROM(z, u):
    # nonlinearity
    fN = np.einsum('ijk,j,k', H_cbal, z, z)
    return f0_cbal + np.dot(A_cbal, z) + fN + np.dot(B_cbal, u)

def cbal_ROM_Jac(z, u):
    J = A_cbal + \
        np.einsum('ijk, k', H_cbal, z) + \
        np.einsum('ijk, j', H_cbal, z)
    
    return J

def cbal_ROM_obs(z):
    return np.dot(C_cbal, z)

## Trajectory predictions

In [ ]:
## predict trajectory

n_traj = len(alphas)
z_cbal_pred = np.inf*np.ones((n_traj, A_cbal.shape[0], len(t_data)))
for k in range(n_traj):
    # simulate the ROM trajectory
    u = sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
    z0 = B_cbal[:,0]*alphas[k]
    sol = sp.integrate.solve_ivp(lambda t,z: cbal_ROM(z, u(t)), \
                                 jac = lambda t,z: cbal_ROM_Jac(z, u(t)), \
                                 t_span = [t_data[0], t_data[-1]], \
                                 y0 = z0, \
                                 t_eval = t_data, \
                                 method = 'BDF')
    
    z_cbal_pred[k,:,:sol.y.shape[1]] = sol.y

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

cbal_sq_errors = np.square(np.linalg.norm(np.einsum('ijl,kj', z_cbal_pred, Phi_cbal) - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

cbal_energy = np.square(np.linalg.norm(np.einsum('ijl,kj', z_cbal_pred, Phi_cbal), axis=1))

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, cbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='cov. bal.')
#ax1.semilogy(t_data, POD_sq_errors[k,:], color='#a6cee3', linestyle=':', label='POD')
for k in range(1, n_traj):
    ax1.semilogy(t_data, cbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    #ax1.semilogy(t_data, POD_sq_errors[k,:], color='#a6cee3', linestyle=':')

ax1.set_ylabel('normalized square error')

ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='lower right')

# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 40
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj)

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_cbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_cbal,z_cbal_pred[traj_idx,:,t_idx]) ))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_cbal, _, _ = output_fields(jet,flowfield_cbal)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_cbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()

# POD

In [ ]:
# X = np.hstack(x_traj_data) / np.sqrt(n_traj * len(t_data))

U_POD, s_POD, VT_POD = np.linalg.svd(X, full_matrices=False, compute_uv=True)
var_frac_POD = np.cumsum(np.square(s_POD))/np.sum(np.square(s_POD))

In [ ]:
plt.figure()
plt.semilogy(np.arange(len(s_POD))+1, 1.0 - var_frac_POD, 'ko')
plt.show()

In [ ]:
r = 40

Phi_POD = U_POD[:,:r]
Psi_POD = U_POD[:,:r]

# construct reduced-order model
f0_POD, A_POD, B_POD, C_POD, H_POD, N_POD = assemble_rom(jet,lops, Phi_POD, Psi_POD)

def POD_ROM(z, u):
    # nonlinearity
    fN = np.einsum('ijk,j,k', H_POD, z, z)
    return f0_POD + np.dot(A_POD, z) + fN + np.dot(B_POD, u)

def POD_ROM_Jac(z, u):
    J = A_POD + \
        np.einsum('ijk, k', H_POD, z) + \
        np.einsum('ijk, j', H_POD, z)
    
    return J

def POD_ROM_obs(z):
    return np.dot(C_POD, z)

## Trajectory predictions

In [ ]:
## predict trajectory

n_traj = len(alphas)
z_POD_pred = np.inf*np.ones((n_traj, A_POD.shape[0], len(t_data)))
for k in range(n_traj):
    # simulate the ROM trajectory
    u = sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
    z0 = B_POD[:,0]*alphas[k]
    sol = sp.integrate.solve_ivp(lambda t,z: POD_ROM(z, u(t)), \
                                 jac = lambda t,z: POD_ROM_Jac(z, u(t)), \
                                 t_span = [t_data[0], t_data[-1]], \
                                 y0 = z0, \
                                 t_eval = t_data, \
                                 method = 'BDF')
    
    z_POD_pred[k,:,:sol.y.shape[1]] = sol.y

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

POD_sq_errors = np.square(np.linalg.norm(np.einsum('ijl,kj', z_POD_pred, Phi_POD) - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

POD_energy = np.square(np.linalg.norm(np.einsum('ijl,kj', z_POD_pred, Phi_POD), axis=1))

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, POD_sq_errors[k,:], color='#a6cee3', linestyle=':', label='POD')
for k in range(1, n_traj):
    ax1.semilogy(t_data, POD_sq_errors[k,:], color='#a6cee3', linestyle=':')

ax1.set_ylabel('normalized square error')

ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='lower right')

# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 30
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj)

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_POD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_POD,z_POD_pred[traj_idx,:,t_idx]) ))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_POD, _, _ = output_fields(jet,flowfield_POD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_POD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()

# BPOD

## Linear forward samples

In [ ]:
# linear impulse response about the steady base flow
X_BPOD, _ = simulate_linear_forward(jet,lops,lops.Mrem.dot(lops.B),time,nsave)

# output projection
U_op, s_op, VT_op = np.linalg.svd(X_BPOD, full_matrices=False, compute_uv=True)
var_frac_op = np.cumsum(np.square(s_op))/np.sum(np.square(s_op))

In [ ]:
# determine output projection rank
plt.figure()
plt.semilogy(np.arange(len(s_op))+1, 1.0 - var_frac_op, 'ko')
plt.show()

## Adjoint samples with output projection

In [ ]:
r_op = 20

Y_BPOD = [np.zeros(X_BPOD.shape) for _ in range(r_op)]
for i in range(r_op):
    print("adjoint sample {:d} of {:d} \n".format(i+1, r_op))
    Y_BPOD[i][:,:], _ = simulate_linear_adjoint(jet,lops,U_op[:,i],time,nsave)

Y_BPOD = np.hstack(Y_BPOD)

## Save BPOD sample matrices

In [ ]:
fname = 'data/BPOD_{:d}op.mat'.format(r_op)
data = {'X_BPOD': X_BPOD, \
        'Y_BPOD': Y_BPOD}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

## Load BPOD sample matrices

In [ ]:
r_op = 20

fname = 'data/BPOD_{:d}op.mat'.format(r_op)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

X_BPOD = data['X_BPOD']
Y_BPOD = data['Y_BPOD']

## Construct projection

In [ ]:
U_BPOD, s_BPOD, VT = sp.linalg .svd(Y_BPOD.T@X_BPOD)
# U_BPOD, s_BPOD, VT = sp.linalg .svd(Y_BPOD.T@X)
V_BPOD = VT.T

var_frac_BPOD = np.cumsum(np.square(s_BPOD))/np.sum(np.square(s_BPOD))

In [ ]:
plt.figure()
plt.semilogy(np.arange(len(s_BPOD))+1, 1.0 - var_frac_BPOD, 'ko')
plt.show()

In [ ]:
r = 40
Phi_BPOD = X_BPOD@V_BPOD[:,:r]/np.sqrt(s_BPOD[:r])
# Phi_BPOD = X@V_BPOD[:,:r]/np.sqrt(s_BPOD[:r])
Psi_BPOD = Y_BPOD@U_BPOD[:,:r]/np.sqrt(s_BPOD[:r])

# construct reduced-order model
f0_BPOD, A_BPOD, B_BPOD, C_BPOD, H_BPOD, N_BPOD = assemble_rom(jet,lops, Phi_BPOD, Psi_BPOD)

def BPOD_ROM(z, u):
    # nonlinearity
    fN = np.einsum('ijk,j,k', H_BPOD, z, z)
    return f0_BPOD + np.dot(A_BPOD, z) + fN + np.dot(B_BPOD, u)

def BPOD_ROM_Jac(z, u):
    J = A_BPOD + \
        np.einsum('ijk, k', H_BPOD, z) + \
        np.einsum('ijk, j', H_BPOD, z)
    
    return J

def BPOD_ROM_obs(z):
    return np.dot(C_BPOD, z)

## Trajectory predictions

In [ ]:
## predict trajectory

n_traj = len(alphas)
z_BPOD_pred = np.inf*np.ones((n_traj, A_BPOD.shape[0], len(t_data)))
for k in range(n_traj):
    # simulate the ROM trajectory
    u = sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
    z0 = B_BPOD[:,0]*alphas[k]
    sol = sp.integrate.solve_ivp(lambda t,z: BPOD_ROM(z, u(t)), \
                                 jac = lambda t,z: BPOD_ROM_Jac(z, u(t)), \
                                 t_span = [t_data[0], t_data[-1]], \
                                 y0 = z0, \
                                 t_eval = t_data, \
                                 method = 'BDF')
    
    z_BPOD_pred[k,:,:sol.y.shape[1]] = sol.y

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

BPOD_sq_errors = np.square(np.linalg.norm(np.einsum('ijl,kj', z_BPOD_pred, Phi_BPOD) - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

BPOD_energy = np.square(np.linalg.norm(np.einsum('ijl,kj', z_BPOD_pred, Phi_BPOD), axis=1))

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, BPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='BPOD')
for k in range(1, n_traj):
    ax1.semilogy(t_data, BPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')

ax1.set_ylabel('normalized square error')

ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='lower right')

# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 10
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj)

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_BPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_BPOD,z_BPOD_pred[traj_idx,:,t_idx]) ))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_BPOD, _, _ = output_fields(jet,flowfield_BPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_BPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()

# TrOOP
Here, we work with orthonormal representatives $\Phi, \Psi\in\mathbb{R}_*^{n,r}$ of the subspaces $V,W \in \mathcal{G}_{n,r}$ used to define the projection $$P_{V,W} = \Phi (\Psi^T\Phi)^{-1}\Psi^T$$

paper:

S. E. Otto, A. Padovan, and C. W. Rowley, ``Optimizing Oblique Projections for Nonlinear Systems using Trajectories'', SIAM Journal on Scientific Computing, Vol. 44, No. 3, 2022

## Validate basic FOM functions

In [ ]:
Phi = Phi_cbal
Psi = Psi_cbal

f0_ROM, A_ROM, B_ROM, C_ROM, H_ROM, N_ROM = assemble_rom(jet,lops, Phi, Psi)

def ROM(z, u):
    f = f0_ROM + np.einsum('ij,...j', A_ROM, z) + np.einsum('ij,...j', B_ROM, u) + \
    np.einsum('ijk,...j,...k', H_ROM, z, z) + np.einsum('ijk,...j,...k', N_ROM, u, z)
    return f

In [ ]:
# z = np.dot(Psi.T, x_traj_data[0,:,0])
z = np.random.randn(r)
u = np.random.randn(1)

x = np.dot(Phi, z)
xdot = rom.evaluate_f(jet,lops,x,u)
zdot = np.dot(Psi.T, xdot)

zdot_ROM = ROM(z, u)

print(np.linalg.norm(zdot_ROM - zdot) / np.linalg.norm(zdot))

In [ ]:
x = x_traj_data[0,:,0]
u = np.random.randn(1)
v = np.random.randn(x.shape[0])

df_v = rom.evaluate_df(jet,lops,x,u,v)

eps = 1.0e-8

df_v_fd = (rom.evaluate_f(jet,lops,x + eps*v,u) - rom.evaluate_f(jet,lops,x,u)) / eps

print(np.linalg.norm(df_v_fd - df_v) / np.linalg.norm(df_v))

In [ ]:
x = np.random.randn(X.shape[0])
u = np.random.randn(1)
v = np.random.randn(x.shape[0])
w = np.random.randn(x.shape[0])

df_v = rom.evaluate_df(jet,lops,x,u,v)
dfT_w = rom.evaluate_df_adj(jet,lops,x,u,w)

IP1 = np.dot(df_v, w)
IP2 = np.dot(v, dfT_w)

print(np.absolute(IP2 - IP1) / np.absolute(IP1))

## Split up trajectories

This was not actually necessary, but here is the code to do it

In [ ]:
L_TrOOP = 100
n_split = len(t_data)//L_TrOOP
n_traj_TrOOP = n_traj*n_split
x_train_TrOOP = np.zeros((n_traj*n_split, x_traj_data.shape[1], L_TrOOP))
for k in range(n_traj):
    for l in range(n_split):
        x_train_TrOOP[n_split*k+l,:,:] = x_traj_data[k,:,l*L_TrOOP:(l+1)*L_TrOOP]

t_TrOOP = t_data[:L_TrOOP]

## Grassmann functions

In [ ]:
def Grassmann_metric(X0, X1, Phi):
    # assemble and factor Gram matrix
    c, low = sp.linalg.cho_factor(np.dot(Phi.T, Phi))
    
    # evaluate metric by solving linear system
    g = np.trace(sp.linalg.cho_solve((c, low), np.dot(X0.T, X1)))
    
    return g

def project_onto_horizontal_subspace(X, Phi):
    # assemble and factor Gram matrices
    c, low = sp.linalg.cho_factor(np.dot(Phi.T, Phi))
    
    # orthogonal projection onto horizontal space
    X_proj = X - np.dot(Phi, sp.linalg.cho_solve((c, low), np.dot(Phi.T, X)))
    
    return X_proj

def Grassmann_exp(Phi, X):
    # exponential map using an orthogonal representative Phi
    # and returning an orthogonal representative.
    # We use Theorem 2.3 in 
    # A. Edelman, T. A. Arias, S. T. Smith, ``The geometry of algorithms with 
    # orthogonality constraints'', SIAM J. Matrix Anal. Appl., 1998.
    U, s, VT = sp.linalg.svd(X, full_matrices=False)
    cos_s = np.reshape(np.cos(s), (1,-1))
    sin_s = np.reshape(np.sin(s), (1,-1))
    
    exp_Phi_X = np.dot(Phi, np.dot(VT.T * cos_s, VT )) + np.dot(U * sin_s, VT)
    return exp_Phi_X

def Grassmann_exp_transport(Phi, X, Y):
    # Transport tangent vector Y along the geodesic starting at Phi
    # in the direction X
    # We use Theorem 2.4 in 
    # A. Edelman, T. A. Arias, S. T. Smith, ``The geometry of algorithms with 
    # orthogonality constraints'', SIAM J. Matrix Anal. Appl., 1998.
    U, s, VT = sp.linalg.svd(X, full_matrices=False)
    cos_s = np.reshape(np.cos(s), (1,-1))
    sin_s = np.reshape(np.sin(s), (1,-1))
    
    # exponential map
    exp_Phi_X = np.dot(Phi, np.dot(VT.T * cos_s, VT )) + np.dot(U * sin_s, VT)
    
    # parallel translation
    T = - np.dot(Phi, VT.T * sin_s) + U * cos_s
    U_Y = np.dot(U.T, Y)
    P_Phi_X_Y = np.dot(T, U_Y) + Y - np.dot(U, U_Y)
    return exp_Phi_X, P_Phi_X_Y

## Optimization objective and gradient

In [ ]:
gamma = 1.0e-3

## parameters for quadrature used to compute gradient
n_quad = 3

# Gauss-Legendre quadrature
s_quad, w_quad = np.polynomial.legendre.leggauss(n_quad)

# set tolerance for solvers
atol = 1.0e-6
rtol = 1.0e-9
solver_method = 'RK45'

# # Trapezoidal quadrature
# s_quad = np.linspace(-1,1,n_quad)
# w_quad = np.zeros(n_quad)
# w_quad[0] = (s_quad[1] - s_quad[0])/2.0
# w_quad[1:-1] = (s_quad[2:] - s_quad[0:-2])/2.0
# w_quad[-1] = (s_quad[-1] - s_quad[-2])/2.0

def regularization(Phi, Psi):
    # regularization and gradient at orthonormal representatives Phi, Psi
    G_Phi = np.dot(Phi.T, Phi)
    G_Psi = np.dot(Psi.T, Psi)
    A_lu, A_piv = sp.linalg.lu_factor(np.dot(Psi.T, Phi))
    
    _, c1 = np.linalg.slogdet(G_Phi)
    _, c2 = np.linalg.slogdet(G_Psi)
    _, c3 = np.linalg.slogdet(np.dot(Psi.T, Phi))
    
    rho = c1 + c2 - 2*c3
    
    Psi_AT = sp.linalg.lu_solve((A_lu, A_piv), Psi.T, trans=0).T
    Phi_A = sp.linalg.lu_solve((A_lu, A_piv), Phi.T, trans=1).T
    
    grad_Phi = 2.0*(Phi - Psi_AT)
    grad_Psi = 2.0*(Psi - Phi_A)
    
    return rho, grad_Phi, grad_Psi

def objective(Phi, Psi):
    r = Phi.shape[1]
    # construct a bi-orthogonalized representative Psi_AT so that
    # P_{V,W} = Phi * Psi_AT^T
    A_lu, A_piv = sp.linalg.lu_factor(np.dot(Psi.T, Phi))
    Psi_AT = sp.linalg.lu_solve((A_lu, A_piv), Psi.T, trans=0).T
    
    # construct a representative of the reduced-order model
    f0_ROM, A_ROM, B_ROM, C_ROM, H_ROM, N_ROM = assemble_rom(jet,lops, Phi, Psi_AT)
    
    def ROM(z, u):
        f = f0_ROM + np.einsum('ij,...j', A_ROM, z) + np.einsum('ij,...j', B_ROM, u) + \
        np.einsum('ijk,...j,...k', H_ROM, z, z) + np.einsum('ijk,...j,...k', N_ROM, u, z)
        return f
    
#     def d_ROM(x, u, v):
#         df_v = np.einsum('ij,...j', A_ROM, v) + \
#             2.0*np.einsum('ijk,...j,...k', H_ROM, x, v) + np.einsum('ijk,...j,...k', N_ROM, u, v)
#         return df_v

    # observation function
    def ROM_obs(z):
        g = np.einsum('ij,...j', C_ROM, z)
        return g
    
    # compute the cost for each trajectory
    try:
        cost = 0.0
        for k in range(n_traj_TrOOP):
            # interpolate the input
            u_fun = lambda t: np.zeros(1)# sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
            
            # initial condition
#             z_0 = B_ROM[:,0]*alphas[k]
            z_0 = np.dot(Psi_AT.T, x_train_TrOOP[k,:,0])
            
            # simulate the ROM trajectory
            sol = sp.integrate.solve_ivp(lambda t,z: ROM(z, u_fun(t)), \
#                                          jac = lambda t,z: d_ROM(z, u_fun(t), np.eye(r)), \
                                         t_span = [t_TrOOP[0], t_TrOOP[-1]], \
                                         y0 = z_0, \
                                         t_eval = t_TrOOP, \
                                         method = solver_method, \
                                         atol=atol, rtol=rtol)
            
            # construct ROM output
            y_ROM = ROM_obs(sol.y.T)
            
            # compute cost along trajectory
            tot_err_k = np.sum(np.square(np.linalg.norm( x_train_TrOOP[k,:,:].T - y_ROM, axis=1)))
            tot_nrgy_k = np.sum(np.square(np.linalg.norm( x_train_TrOOP[k,:,:].T, axis=1)))
            cost = cost + tot_err_k / tot_nrgy_k
        
        if np.isnan(cost):
            cost = np.inf
        
    except:
        cost = np.inf
    
    rho, _, _ = regularization(Phi, Psi)
    
    return cost/n_traj_TrOOP + gamma*rho

def objective_grad(Phi, Psi):
    r = Phi.shape[1]
    # construct a bi-orthogonalized representative Psi_AT so that
    # P_{V,W} = Phi * Psi_AT^T
    A_lu, A_piv = sp.linalg.lu_factor(np.dot(Psi.T, Phi))
    Psi_AT = sp.linalg.lu_solve((A_lu, A_piv), Psi.T, trans=0).T
    
    # construct a representative of the reduced-order model
    f0_ROM, A_ROM, B_ROM, C_ROM, H_ROM, N_ROM = assemble_rom(jet,lops, Phi, Psi_AT)
    
    def ROM(z, u):
        f = f0_ROM + np.einsum('ij,...j', A_ROM, z) + np.einsum('ij,...j', B_ROM, u) + \
        np.einsum('ijk,...j,...k', H_ROM, z, z) + np.einsum('ijk,...j,...k', N_ROM, u, z)
        return f
    
#     def d_ROM(x, u, v):
#         df_v = np.einsum('ij,...j', A_ROM, v) + \
#             2.0*np.einsum('ijk,...j,...k', H_ROM, x, v) + np.einsum('ijk,...j,...k', N_ROM, u, v)
#         return df_v

    # observation function
    def ROM_obs(z):
        g = np.einsum('ij,...j', C_ROM, z)
        return g
    
    # adjoint of ROM acting on w
    def d_ROM_adj(z, u, w):
        df_T_w = np.einsum('ji,...j', A_ROM, w) + \
            2.0*np.einsum('kji,...j,...k', H_ROM, z, w) + np.einsum('kji,...j,...k', N_ROM, u, w)
        return df_T_w
    
    # adjoint of ROM observations acting on w
    def d_ROM_obs_adj(z, w):
        dg_T_w = np.einsum('ji,...j', C_ROM, w)
        return dg_T_w
    
    ## compute cost and gradient
    cost = 0.0
    grad_Phi = 0.0 * Phi
    grad_Psi = 0.0 * Psi
    
#     for k in range(n_traj):
    for k in tqdm(range(n_traj_TrOOP), position=0, leave=True, desc="computing gradient"):
        # interpolate the input
        u_fun = lambda t: np.zeros(1)# sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
        
        # initial condition
        x_0 = x_train_TrOOP[k,:,0]
        z_0 = np.dot(Psi_AT.T, x_0) # B_ROM[:,0]*alphas[k]
        
        # simulate the ROM trajectory
        sol = sp.integrate.solve_ivp(lambda t,z: ROM(z, u_fun(t)), \
#                                      jac = lambda t,z: d_ROM(z, u_fun(t), np.eye(r)), \
                                     t_span = [t_TrOOP[0], t_TrOOP[-1]], \
                                     y0 = z_0, \
                                     dense_output=True, \
                                     method = solver_method, \
                                     atol=atol, rtol=rtol)
        
        # interpolants of the ROM solution
        z_fun = lambda t: sol.sol(t).T
        y_fun = lambda t: ROM_obs(sol.sol(t).T)
        
        # outputs at sample times
        y_ROM = y_fun(t_TrOOP)
        
        # compute cost
        tot_err_k = np.sum(np.square(np.linalg.norm( x_train_TrOOP[k,:,:].T - y_ROM, axis=1)))
        tot_nrgy_k = np.sum(np.square(np.linalg.norm( x_train_TrOOP[k,:,:].T, axis=1)))
        cost = cost + tot_err_k / tot_nrgy_k
        
        # cost gradient at each sample time
        g_samples = -2.0 * (x_train_TrOOP[k,:,:].T - y_ROM) / tot_nrgy_k
        
        # construct F, S, H, T operators using prop. 4.3
        F_adj = lambda t, v: rom. d_ROM_adj(z_fun(t), u_fun(t), v)
        S_adj_Phi = lambda t, v: np.outer(rom.evaluate_df_adj(jet,lops,np.dot(Phi, z_fun(t)), \
                                                     u_fun(t), \
                                                     np.dot(Psi_AT, v)), \
                                          z_fun(t)) - \
                                np.outer(np.dot(Psi_AT, v), \
                                         ROM(z_fun(t), u_fun(t)))
        S_adj_Psi = lambda t, v: np.outer(rom.evaluate_f(jet,lops,np.dot(Phi, z_fun(t)), u_fun(t)) - \
                                          np.dot(Phi, ROM(z_fun(t), u_fun(t))), \
                                          sp.linalg.lu_solve((A_lu, A_piv), v, trans=1).T)
        H_adj = lambda t, v: d_ROM_obs_adj(z_fun(t), v)
        T_adj_Phi = lambda t, v: np.outer(v, z_fun(t))
        T_adj_Psi = lambda t, v: 0.0*Psi
        IC_adj_Phi = lambda v: -np.outer(np.dot(Psi_AT,v), z_0)
        IC_adj_Psi = lambda v: np.outer(x_0 - np.dot(Phi, z_0), \
                                        sp.linalg.lu_solve((A_lu, A_piv), v, trans=1).T)
        
        # add last element of sum component of gradient
        tf = t_TrOOP[-1]
        grad_Phi = grad_Phi + T_adj_Phi(tf, g_samples[-1,:])
        grad_Psi = grad_Psi + T_adj_Psi(tf, g_samples[-1,:])
        
        # solve adjoint equation with jumps backwards in time
        lam_f = H_adj(t_TrOOP[-1], g_samples[-1,:])
        for l in range(len(t_TrOOP)-1):
            # simulate adjoint dynamics (thm. 4.2)
            # over the interval t_data[-2-l] to t_data[-1-l] with
            # tau = t_data[-1] - t
            
            # sample points for quadrature
            s_pts =  t_TrOOP[-2-l] + (t_TrOOP[-1-l] - t_TrOOP[-2-l])*(s_quad + 1.0)/2.0
            tau_pts = tf - s_pts
            
            # scaled quadrature weights
            w_scaled = w_quad * (t_TrOOP[-1-l] - t_TrOOP[-2-l])/2.0
            
            # integrate adjoint dynamics
            adj_sol = sp.integrate.solve_ivp(lambda tau,lam: d_ROM_adj(z_fun(tf-tau), u_fun(tf-tau),lam), \
#                                     jac = lambda tau,lam: d_ROM_adj(z_fun(tf-tau), u_fun(tf-tau),np.eye(r)), \
                                    t_span = [tf-t_TrOOP[-1-l], tf-t_TrOOP[-2-l]], \
                                    y0 = lam_f, \
                                    dense_output=True, \
                                    method = solver_method, \
                                    atol=atol, rtol=rtol)
            
            # compute adjoint variable at quadrature points s_pts
            lam = adj_sol.y[:,::-1].T
            
            # add integral component to gradient
            for q in range(n_quad):
                grad_Phi = grad_Phi + w_scaled[q]*S_adj_Phi(s_pts[q], adj_sol.sol(tau_pts[q]).T)
                grad_Psi = grad_Psi + w_scaled[q]*S_adj_Psi(s_pts[q], adj_sol.sol(tau_pts[q]).T)
            
            # add element to sum component of gradient
            grad_Phi = grad_Phi + T_adj_Phi(t_TrOOP[-2-l], g_samples[-2-l,:])
            grad_Psi = grad_Psi + T_adj_Psi(t_TrOOP[-2-l], g_samples[-2-l,:])
            
            # add jump to adjoint variable
            lam_f = adj_sol.sol(tf-t_TrOOP[-2-l]).T + H_adj(t_TrOOP[-2-l], g_samples[-2-l,:])
        
        # add gradient due to initial condition
        grad_Phi = grad_Phi + IC_adj_Phi(lam_f)
        grad_Psi = grad_Psi + IC_adj_Psi(lam_f)
        
    # compute regularization
    rho, grad_rho_Phi, grad_rho_Psi = regularization(Phi, Psi)
    
    # compute normalized and regularized cost and gradient
    cost = cost/n_traj_TrOOP + gamma*rho
    grad_Phi = grad_Phi/n_traj_TrOOP + gamma*grad_rho_Phi
    grad_Psi = grad_Psi/n_traj_TrOOP + gamma*grad_rho_Psi
        
    return cost, grad_Phi, grad_Psi

In [ ]:
# choose subspaces
Phi, _, _ = sp.linalg.qr(Phi_cbal, pivoting=True, mode='economic')
Psi, _, _ = sp.linalg.qr(Psi_cbal, pivoting=True, mode='economic')

# make sure orientation is correct
s, _ = np.linalg.slogdet(np.dot(Psi.T, Phi))
Phi[:,0] = s*Phi[:,0]

# initial objective value and gradient
f_val, grad_Phi, grad_Psi = objective_grad(Phi, Psi)
print(f_val)

# magnitude of the gradient
grad_mag = np.sqrt(Grassmann_metric(grad_Phi, grad_Phi, Phi) + Grassmann_metric(grad_Psi, grad_Psi, Psi))
print(grad_mag)

### Validate gradient

In [ ]:
# random perturbation in tangent space
# X = project_onto_horizontal_subspace(np.random.randn(Phi.shape[0], Phi.shape[1])/np.sqrt(Phi.shape[0]), Phi)
# Y = project_onto_horizontal_subspace(np.random.randn(Psi.shape[0], Psi.shape[1])/np.sqrt(Psi.shape[0]), Psi)

X = project_onto_horizontal_subspace(grad_Phi/grad_mag, Phi)
Y = project_onto_horizontal_subspace(grad_Psi/grad_mag, Psi)

# derivative along (X,Y) as computed using gradient
dfdt_grad = Grassmann_metric(grad_Phi, X, Phi) + Grassmann_metric(grad_Psi, Y, Psi)
print(dfdt_grad)

# apply exponential map to construct nearby point
eps = 1.0e-10
Phi_eps = Grassmann_exp(Phi, eps*X)
Psi_eps = Grassmann_exp(Psi, eps*Y)
f_eps = objective(Phi_eps, Psi_eps)

# derivative using finite differences
dfdt_fd = (f_eps - f_val)/eps
print(dfdt_fd)

print(np.absolute((dfdt_grad - dfdt_fd))/np.absolute(dfdt_fd))

## Conjugate gradient functions

In [ ]:
def line_search_objective(step, Phi, Psi, dir_X, dir_Y):
    Phi_next = Grassmann_exp(Phi, step*dir_X)
    Psi_next = Grassmann_exp(Psi, step*dir_Y)
    f = objective(Phi_next, Psi_next)
    return f

def weak_Wolfe_bisection(slope, init_step_size, c1, c2, f0, f, fd_step, \
    max_iter=np.inf):
    """ Bisection line search method to meet weak Wolfe conditions descibed in
    https://sites.math.washington.edu/~burke/crs/408/notes/nlp/line.pdf.
    The derivative along the search direction is computed using finite
    differences.

    Parameters
    ----------
    slope: float
        The slope along the search direction, i.e., the directional derivative 
        of the objective along the search direction.
    init_step_size: float
        A positive number giving the initial step size to consider along the
        search direction.
    c1: float
        A number satisfying 0 < c1 < c2 < 1 giving the desired fraction of the 
        initial slope to be achieved by the average decrease of the objective 
        along the step.
    c2: float
        A number satisfying 0 < c1 < c2 < 1 giving the desired fraction by 
        which the slope along the search direction should increase.
    f0: float
        The value of the objective at the current step of the outer algorithm.
    f: function
        A function of the step size that retuns the value of the objective when
        a step of a given size is taken. This function satisfies f0 = f(0) and 
        f'(0) = slope.
    fd_step: float
        A small positive number to be used as the step size for estimating the
        derivative along the search direction by finite differences. If the
        step_size is smaller that 10*fd_step, we use 0.1*fd_step for the finite
        difference.
    max_iter: integer
        The maximum number of bisection iterations to perform. The default is
        infinity since the algorithm is guaranteed to eventually converge.
    
    Returns
    -------
    step_size: float
        A step size meeting the weak Wolfe conditions (assuming the max_iter
        is not reached).
    f_next: float
        The value of the objective after a step of the chosen size along the
        search direction.
    iter: integer
        The number of iterations use to compute the step size.
    """
    # desired fraction of slope along descent direction to be achieved by step
    # this is the first weak Wolfe condition.
    decrease_cond = -c1*slope

    # maximum rate of decrease to meet the second weak Wolfe condition
    curvature_cond = -c2*slope

    # Initialization
    alpha = 0
    beta = np.inf
    step_size = init_step_size

    iter = 0
    weak_Wolfe_cond = False
    while iter < max_iter and not weak_Wolfe_cond:
        # compute the objective value using the proposed step size
        f_next = f(step_size)

        if f0 - f_next < step_size * decrease_cond:
            # sufficient decrease condition not met, decrease the step size.
            beta = step_size
            step_size = (alpha + beta)/2.0
        else:
            # sufficient decrease condition is met. Check curvature condition by
            # computing the slope at the proposed step.
            h = -np.min((fd_step, step_size/10.0))
            df_next = (f(step_size + h) - f_next) / h

            if -df_next > curvature_cond:
                # rate of objective decrease is too high for second weak Wolfe
                # condition. Adjust the step size.
                alpha = step_size

                if beta == np.inf:
                    step_size = 2.0 * alpha
                else:
                    step_size = (alpha + beta)/2.0
            
            else:
                # the weak Wolfe conditions are met, so iteration stops
                weak_Wolfe_cond = True
        
        iter = iter + 1
    
    return step_size, f_next, iter

def cg_step(Phi, Psi, f_val, dir_X, dir_Y, grad_X, grad_Y, init_step_size, \
    ls_params={'max_iter':np.inf, 'c1':0.1, 'c2':0.2, 'fd_step':1.0e-6}):
    """One step of the geometric conjugate gradient algorithm using bisection
    line search to meet the weak Wolfe conditions. Next search direction is
    computed using the Riemannian Dai-Yuan method of
    
    H. Sato, "A Dai-Yuan-type Riemannian conjugate gradient method with the
    weak Wolfe conditions", 2016, Comput. Optim. and Appl.

    Parameters
    ----------
    Phi: array of shape (n_dim, r_basis)
        Representative of the subspace V, i.e., the columns of Phi are an 
        orthonormal basis for V.
    Psi: array of shape (n_dim, r_basis)
        Representative of the subspace W, i.e., the columns of Psi are an 
        orthonormal basis for W.
    f_val: float
        Current value of the objective function
    dir_X: array of shape (n_dim, r_basis)
        Horizontal lift of the search direction for V in the tangent space
        at Phi. In particular Phi^*dir_X = 0.
    dir_Y: array of shape (n_dim, r_basis)
        Horizontal lift of the search direction for W in the tangent space
        at Psi. In particular Phi^*dir_X = 0.
    grad_X: array of shape (n_dim, r_basis)
        Horizontal lift of gradient wrt V at Phi.
    grad_Y: array of shape (n_dim, r_basis)
        Horizontal lift of gradient wrt W at Psi
    init_step_size: float
        A positive number giving the initial step size along the search 
        direction.
    ls_params: dictionary
        Parameters for the line search method. Includes:
            max_iter: integer
                The maximum number of bisection iterations to perform.
            c1: float
                A number satisfying 0 < c1 < c2 < 1 giving the desired fraction 
                of the initial slope to be achieved by the average decrease of 
                the objective along the step.
            c2: float
                A number satisfying 0 < c1 < c2 < 1 giving the desired fraction 
                by which the slope along the search direction should increase.
            fd_step: float
                Step size used to compute the derivative along the line search
                direction. When step_size < 10*fd_step, we reduce this value to 
                0.1*step_size.

    Returns
    -------
    Phi_next: array of shape (n_dim, r_basis)
        Orthonormal representative of V at the next step
    Psi_next: array of shape (n_dim, r_basis)
        Orthonormal representative of W at the next step  
    dir_X_next: array of shape (n_dim, r_basis)
        Horizontal lift of the next search direction for V at Phi_next
    dir_Y_next: array of shape (n_dim, r_basis)
        Horizontal lift of the next search direction for W at Psi_next
    grad_X_next: array of shape (n_dim, r_basis)
        Horizontal lift of gradient wrt V at Phi_next
    grad_Y_next: array of shape (n_dim, r_basis)
        Horizontal lift of gradient wrt W at Psi_next
    step_size: float
        The step size computed using the bisection method to meet the weak 
        Wolfe conditions.
    f_val_next: float
        The value of the objective at (Phi_next, Phi_next).
    iter: integer
        The number of iterations used to find the step size.
    """
    # normalize the step size
    #   in particular, the user works with an absolute steps size, whereas this
    #   algorith uses a relative step size that multiplies dir_X and dir_Y
    dir_mag = np.sqrt(Grassmann_metric(dir_X, dir_X, Phi) + \
                      Grassmann_metric(dir_Y, dir_Y, Psi))
    init_step_size = init_step_size/dir_mag

    ## compute step size along search direction using bisection method to meet
    # the weak Wolfe conditions

    # compute derivative of objective along line search direction
    slope = Grassmann_metric(grad_X, dir_X, Phi) + \
                Grassmann_metric(grad_Y, dir_Y, Psi)

    # objective function along line search direction
    f_line_search = lambda step : line_search_objective(step, Phi, Psi, \
                                                        dir_X, dir_Y)
    
    # compute the step size and next objective value
    step_size, f_val_next, iter = weak_Wolfe_bisection(slope, \
            init_step_size, ls_params['c1'], ls_params['c2'], f_val, \
            f_line_search, ls_params['fd_step']/dir_mag, ls_params['max_iter'])
    
    ## compute next iterate and transport search direction to new point
    Phi_next, T_dir_X = Grassmann_exp_transport(Phi, step_size*dir_X, dir_X)
    Psi_next, T_dir_Y = Grassmann_exp_transport(Psi, step_size*dir_Y, dir_Y)

    ## compute the gradient at the next step
    
    # make sure that orientation is correct
    s, _ = np.linalg.slogdet(np.dot(Psi_next.T, Phi_next))
    Phi_next[:,0] = s * Phi_next[:,0]
    T_dir_X[:,0] = s * T_dir_X[:,0]
    
    # compute gradient
    _, grad_X_next, grad_Y_next = objective_grad(Phi_next, Psi_next)

    # Project the gradient onto the horizontal subspace at the next point.
    # This should not do anything because the gradient should already lie in
    # the horizontal space. But we keep to ensure this holds to high numerical
    # precision.
    grad_X_next = project_onto_horizontal_subspace(grad_X_next, Phi_next)
    grad_Y_next = project_onto_horizontal_subspace(grad_Y_next, Psi_next)
    
    ## compute the next search direction
    
    # compute components of the combination coefficient
    a = Grassmann_metric(grad_X_next, grad_X_next, Phi_next) + \
            Grassmann_metric(grad_Y_next, grad_Y_next, Psi_next)
    b = Grassmann_metric(grad_X_next, T_dir_X, Phi_next) + \
            Grassmann_metric(grad_Y_next, T_dir_Y, Psi_next)
    c = Grassmann_metric(grad_X, dir_X, Phi) + \
            Grassmann_metric(grad_Y, dir_Y, Psi)
    
    # combination coefficient
    beta = a / (b - c)
    
    # horizontal lift of the next search direction
    dir_X_next = -grad_X_next + beta*T_dir_X
    dir_Y_next = -grad_Y_next + beta*T_dir_Y
    
    # convert back to absolute step size
    step_size = step_size * dir_mag
    
    return Phi_next, Psi_next, dir_X_next, dir_Y_next, grad_X_next, \
        grad_Y_next, step_size, f_val_next, iter

## Optimization

### Optimization parameters

In [ ]:
# parameters of CG algorithm
init_step_size = 1.0e-2

ls_params = {'max_iter':20, 'c1':0.01, 'c2':0.1, 'fd_step':1.0e-8}

step_size = init_step_size

### Initialization

In [ ]:
Phi, _, _ = sp.linalg.qr(Phi_cbal, pivoting=True, mode='economic')
Psi, _, _ = sp.linalg.qr(Psi_cbal, pivoting=True, mode='economic')

s, _ = np.linalg.slogdet(np.dot(Psi.T, Phi))
Phi[:,0] = s*Phi[:,0]

Phi_init = np.copy(Phi)
Psi_init = np.copy(Psi)

# initial cost, gradient, and search direction
f_val, grad_Phi, grad_Psi = objective_grad(Phi, Psi)
dir_Phi = -grad_Phi
dir_Psi = -grad_Psi
print(f_val)

f_val_history = [f_val]

grad_mag = np.sqrt(Grassmann_metric(grad_Phi, grad_Phi, Phi) + Grassmann_metric(grad_Psi, grad_Psi, Psi))
print(grad_mag)

grad_mag_history = [grad_mag]

### Conjugate gradient iteration

In [ ]:
f_diff = np.inf
iter = 0
#while f_diff > 1.0e-12 and iter < 10:
continue_cond = True

fname = 'data/TrOOP/tmp_TrOOP_iter{:d}.mat'.format(iter)
data = {'Phi': Phi, \
        'Psi': Psi, \
        'f_val': f_val, \
        'grad_mag': grad_mag, \
        'grad_Phi': grad_Phi, \
        'grad_Psi': grad_Psi, \
        'dir_Phi': dir_Phi, \
        'dir_Psi': dir_Psi, \
        'f_val_history': f_val_history, \
        'grad_mag_history': grad_mag_history, \
        'ls_params': ls_params, \
        'iter': iter}
        

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

In [ ]:
continue_cond = True
while iter < 500 and continue_cond:
    
    if step_size < 1.0e-10:
        step_size = init_step_size
    
    Phi_next, Psi_next, dir_Phi_next, dir_Psi_next, grad_Phi_next, \
    grad_Psi_next, step_size, f_val_next, n_steps = cg_step(Phi, Psi, f_val, \
                                                    dir_Phi, dir_Psi, \
                                                    grad_Phi, grad_Psi, \
                                                    step_size, \
                                                    ls_params=ls_params)
    Phi = Phi_next
    Psi = Psi_next
    dir_Phi = dir_Phi_next
    dir_Psi = dir_Psi_next
    grad_Phi = grad_Phi_next
    grad_Psi = grad_Psi_next
    
    grad_mag = np.sqrt(Grassmann_metric(grad_Phi, grad_Phi, Phi) + \
                       Grassmann_metric(grad_Psi, grad_Psi, Psi))
    
    continue_cond = grad_mag > 1.0e-4
    
    print('step {:d} cost {:.3e} grad mag {:.3e} step size {:.3e} LS iters {:d} \n'.format(iter+1, \
                                                                                           f_val_next, \
                                                                                           grad_mag, \
                                                                                           step_size, \
                                                                                           n_steps))
    f_diff = f_val - f_val_next
    f_val = f_val_next
    
    f_val_history = f_val_history + [f_val]
    grad_mag_history = grad_mag_history + [grad_mag]
    
    ## save progress
    if iter % 1 == 0:
        fname = 'data/TrOOP/POD_TrOOP_iter{:d}.mat'.format(iter)
        data = {'Phi': Phi, \
                'Psi': Psi, \
                'f_val': f_val, \
                'grad_mag': grad_mag, \
                'grad_Phi': grad_Phi, \
                'grad_Psi': grad_Psi, \
                'dir_Phi': dir_Phi, \
                'dir_Psi': dir_Psi, \
                'f_val_history': f_val_history, \
                'grad_mag_history': grad_mag_history, \
                'ls_params': ls_params, \
                'iter': iter}
        
        with open(fname, 'wb') as fh:
           pickle.dump(data, fh)
    
    iter = iter + 1

## TrOOP ROM

In [ ]:
fname = 'data/TrOOP/cbal_TrOOP_iter{:d}.mat'.format(499)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

Phi = data['Phi']
Psi = data['Psi']
f_val = data['f_val']
grad_mag = data['grad_mag']
grad_Phi = data['grad_Phi']
grad_Psi = data['grad_Psi']
dir_Phi = data['dir_Phi']
dir_Psi = data['dir_Psi']
f_val_history = data['f_val_history']
grad_mag_history = data['grad_mag_history']
iter = data['iter']

In [ ]:
fname = 'data/TrOOP/POD_TrOOP_iter{:d}.mat'.format(499)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

f_val_history_POD = data['f_val_history']
grad_mag_history_POD = data['grad_mag_history']

In [ ]:
# plt.figure(figsize=(8, 4))
plt.figure()
plt.loglog(np.arange(len(f_val_history))+1, f_val_history, color='#1f78b4', linestyle='-', label='CoBRAS init.')
plt.loglog(np.arange(len(f_val_history_POD))+1, f_val_history_POD, color='#b2df8a', linestyle='--', label='POD init.')

plt.ylim([0.1, 50])

# plt.grid(True, which='both')

plt.legend(loc='upper right')

# plt.savefig('../figures/jet_flow/TrOOP_cost_POD_vs_CoBRAS_init.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/TrOOP_cost_POD_vs_CoBRAS_init.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()



# plt.figure(figsize=(8, 4))
plt.figure()
plt.loglog(np.arange(len(grad_mag_history))+1, grad_mag_history, color='#1f78b4', linestyle='-', label='CoBRAS init.')
plt.loglog(np.arange(len(grad_mag_history_POD))+1, grad_mag_history_POD, color='#b2df8a', linestyle='--', label='POD init.')

#plt.ylim([0.1, 50])

plt.legend(loc='upper right')

# plt.savefig('../figures/jet_flow/TrOOP_grad_POD_vs_CoBRAS_init.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/TrOOP_grad_POD_vs_CoBRAS_init.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
# bi-orthogonalize the matrix representatives
Phi_TrOOP = np.copy(Phi)
Psi_TrOOP = np.linalg.solve(np.dot(Psi.T, Phi), Psi.T).T

# construct reduced-order model
f0_TrOOP, A_TrOOP, B_TrOOP, C_TrOOP, H_TrOOP, N_TrOOP = assemble_rom(jet,lops, Phi_TrOOP, Psi_TrOOP)

def TrOOP_ROM(z, u):
    # nonlinearity
    fN = np.einsum('ijk,j,k', H_TrOOP, z, z)
    return f0_TrOOP + np.dot(A_TrOOP, z) + fN + np.dot(B_TrOOP, u)

def TrOOP_ROM_Jac(z, u):
    J = A_TrOOP + \
        np.einsum('ijk, k', H_TrOOP, z) + \
        np.einsum('ijk, j', H_TrOOP, z)
    
    return J

def TrOOP_ROM_obs(z):
    return np.dot(C_TrOOP, z)

## Trajectory predictions

In [ ]:
## predict trajectory

n_traj = len(alphas)
z_TrOOP_pred = np.inf*np.ones((n_traj, A_TrOOP.shape[0], len(t_data)))
for k in range(n_traj):
    # simulate the ROM trajectory
    u = sp.interpolate.interp1d(time, u_traj[k,:,:], axis=1)
    z0 = B_TrOOP[:,0]*alphas[k]
    sol = sp.integrate.solve_ivp(lambda t,z: TrOOP_ROM(z, u(t)), \
                                 jac = lambda t,z: TrOOP_ROM_Jac(z, u(t)), \
                                 t_span = [t_data[0], t_data[-1]], \
                                 y0 = z0, \
                                 t_eval = t_data, \
                                 method = 'BDF')
    
    z_TrOOP_pred[k,:,:sol.y.shape[1]] = sol.y

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

TrOOP_sq_errors = np.square(np.linalg.norm(np.einsum('ijl,kj', z_TrOOP_pred, Phi_TrOOP) - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

TrOOP_energy = np.square(np.linalg.norm(np.einsum('ijl,kj', z_TrOOP_pred, Phi_TrOOP), axis=1))

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, TrOOP_sq_errors[k,:], color='#33a02c', linestyle='-', label='TrOOP')
for k in range(1, n_traj):
    ax1.semilogy(t_data, TrOOP_sq_errors[k,:], color='#33a02c', linestyle='-')

ax1.set_ylabel('normalized square error')

ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='lower right')

# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# Compare Galerkin predictions

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, cbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='CoBRAS')
ax1.semilogy(t_data, TrOOP_sq_errors[k,:], color='#33a02c', linestyle='-', label='TrOOP')
ax1.semilogy(t_data, POD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='POD')
ax1.semilogy(t_data, BPOD_sq_errors[k,:], color='#a6cee3', linestyle=':', label='BPOD')
for k in range(1, n_traj):
    ax1.semilogy(t_data, cbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, TrOOP_sq_errors[k,:], color='#33a02c', linestyle='-')
    ax1.semilogy(t_data, POD_sq_errors[k,:], color='#b2df8a', linestyle='--')
    ax1.semilogy(t_data, BPOD_sq_errors[k,:], color='#a6cee3', linestyle=':')

# ax1.set_ylabel('normalized square error')

# ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='lower right')

# plt.savefig('../figures/jet_flow/test_data_normalized_square_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_normalized_square_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 10
print(t_data[t_idx])
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj)

# print(alphas[traj_idx])

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_cbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_cbal,z_cbal_pred[traj_idx,:,t_idx]) ))) + q_sbf
flowfield_TrOOP = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_TrOOP,z_TrOOP_pred[traj_idx,:,t_idx]) ))) + q_sbf
flowfield_POD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_POD,z_POD_pred[traj_idx,:,t_idx]) ))) + q_sbf
flowfield_BPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot( np.dot(Phi_BPOD,z_BPOD_pred[traj_idx,:,t_idx]) ))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_cbal, _, _ = output_fields(jet,flowfield_cbal)
_, _, w_TrOOP, _, _ = output_fields(jet,flowfield_TrOOP)
_, _, w_POD, _, _ = output_fields(jet,flowfield_POD)
_, _, w_BPOD, _, _ = output_fields(jet,flowfield_BPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=5, ncols=1, sharex=True)

c=axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_cbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[2].contourf(Z,R,w_TrOOP,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[3].contourf(Z,R,w_POD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[4].contourf(Z,R,w_BPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)
axs[2].set_aspect(1)
axs[3].set_aspect(1)
axs[4].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])
axs[2].set_ylim([0,1.5])
axs[3].set_ylim([0,1.5])
axs[4].set_ylim([0,1.5])

axs[0].set_title('t = {:.2f}, alpha = {:.4f}'.format(t_data[t_idx], alphas[traj_idx]))
# axs[0].set_title('ground truth')
# axs[1].set_title('SGBPOD')
# axs[2].set_title('POD')
# axs[3].set_title('BPOD')

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()


# plt.savefig('../figures/jet_flow/snapshot_preds_t5_Emax.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/snapshot_preds_t5_Emax.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)